In [ ]:
from allennlp_models.pretrained import load_predictor

In [ ]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [ ]:
from checklist.pred_wrapper import PredictorWrapper

In [ ]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [ ]:
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("John met a friend and smiled.")
output

In [ ]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [ ]:
d = ["John met a friend and smiled."]
pred = predict_and_conf(d)
pred

In [ ]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [ ]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [ ]:
# e = get_arg(pred,arg_target='ARG1')
# print(e)

In [ ]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [ ]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name'], meta['last_name']])
    arg_0 = get_arg(pred, arg_target='ARG1')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg1_people)

In [ ]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("John met {first_name} {last_name} and smiled.", meta=True, nsamples=1000, remove_duplicates= True)

#print(type(t))

for k, v in t.items():
    print(k, v)

In [ ]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

In [ ]:
import sys
#[https://howtodoinjava.com/examples/python-print-to-file/]
# Saving the reference of the standard output
original_stdout = sys.stdout  
 
with open('bert_result_items_argument_ellipsis.txt', 'w') as f:
    sys.stdout = f 
    for k, v in test.results.items():
        print(k, v, file = f)
    # Reset the standard output
    sys.stdout = original_stdout 